In [ ]:
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import meshio, numpy, copy
import meshplot as mp
# import sys
# import pathlib
# sys.path.append(str(pathlib.Path("").resolve().parent / "build" / "release" / "python"))  # noqa

from ipctk import *

In [ ]:
V = np.array([[0.5,0],[1.0,0],[1.5,0],[0.9,0.3],[1,0.1],[1.1,0.3],[1,-0.5]])
E = np.array([[1,0],[2,1],[3,4],[4,5],[5,3],[0,6],[6,2]], dtype=int)
F = np.array([])

p = mp.plot(V, F, shading={"width": 200, "height": 200})
p.add_points(V, shading={"point_color": "green", "point_size": 0.1})
p.add_lines(V[E[:,0],:],V[E[:,1],:], shading={"line_width": 2})

dhat=0.12
param = ParameterType(dhat, 1, 0, 1, 0, 1, 200)
mesh = CollisionMesh(V, E, F)

In [ ]:
# high order

def potential_high_order(points):
    cs1 = SmoothCollisions2()
    cs1.use_high_order_quadrature = True
    cs1.build(mesh, points, param)

    B = SmoothPotential(param)
    # g = B.gradient(cs1, mesh, points)

    return B(cs1, mesh, points)

In [ ]:
cs1 = SmoothCollisions2()
cs1.use_high_order_quadrature = True
cs1.build(mesh, V, param)
print(cs1.to_string(mesh, V, param))
print(mesh.edges)

In [ ]:
# single point

def potential_single_point(points):
    cs2 = SmoothCollisions2()
    cs2.use_high_order_quadrature = False
    cs2.build(mesh, points, param)
    
    B = SmoothPotential(param)
    # g = B.gradient(cs1, mesh, points)

    return B(cs2, mesh, points)

In [ ]:
# IPC

def potential_IPC(points):
    cs3 = Collisions()
    cs3.build(mesh, points, param.dhat)

    B = BarrierPotential(param.dhat)
    # g = B.gradient(cs3, mesh, points)

    return B(cs3, mesh, points)

In [ ]:
print(potential_IPC(V), potential_single_point(V), potential_high_order(V))

In [ ]:
displacements = np.linspace(-0.1, 0.1, 200)
pA = []
pB = []
pC = []

for disp in displacements:
    V_disp = copy.deepcopy(V)
    V_disp[[3,4,5], 0] += disp
    
    pA.append(potential_IPC(V_disp))
    pB.append(potential_single_point(V_disp))
    pC.append(potential_high_order(V_disp))


In [ ]:
fig = go.Figure(data=[
    go.Scatter(x=displacements, y=pA, name="IPC"),
    go.Scatter(x=displacements, y=pB, name="Single Point"),
    go.Scatter(x=displacements, y=pC, name="Exact Quadrature"),
], layout=go.Layout(width=800, height=400))

# fig.update_layout(
#     yaxis_type="log"
# )

fig.show()